In [132]:
import pandas as pd
import numpy as np
import re
import gensim
import pickle
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import word_tokenize
from os import listdir
from os.path import isfile, join
from gensim.models.deprecated.doc2vec import LabeledSentence

In [133]:
countries = [
{'timezones': ['Europe/Andorra'], 'code': 'AD', 'continent': 'Europe', 'name': 'Andorra', 'capital': 'Andorra la Vella'},
{'timezones': ['Asia/Kabul'], 'code': 'AF', 'continent': 'Asia', 'name': 'Afghanistan', 'capital': 'Kabul'},
{'timezones': ['America/Antigua'], 'code': 'AG', 'continent': 'north-america', 'name': 'Antigua and Barbuda', 'capital': "St. John's"},
{'timezones': ['Europe/Tirane'], 'code': 'AL', 'continent': 'Europe', 'name': 'Albania', 'capital': 'Tirana'},
{'timezones': ['Asia/Yerevan'], 'code': 'AM', 'continent': 'Asia', 'name': 'Armenia', 'capital': 'Yerevan'},
{'timezones': ['Africa/Luanda'], 'code': 'AO', 'continent': 'Africa', 'name': 'Angola', 'capital': 'Luanda'},
{'timezones': ['America/Argentina/Buenos_Aires', 'America/Argentina/Cordoba', 'America/Argentina/Jujuy', 'America/Argentina/Tucuman', 'America/Argentina/Catamarca', 'America/Argentina/La_Rioja', 'America/Argentina/San_Juan', 'America/Argentina/Mendoza', 'America/Argentina/Rio_Gallegos', 'America/Argentina/Ushuaia'], 'code': 'AR', 'continent': 'south-america', 'name': 'Argentina', 'capital': 'Buenos Aires'},
{'timezones': ['Europe/Vienna'], 'code': 'AT', 'continent': 'Europe', 'name': 'Austria', 'capital': 'Vienna'},
{'timezones': ['Australia/Lord_Howe', 'Australia/Hobart', 'Australia/Currie', 'Australia/Melbourne', 'Australia/Sydney', 'Australia/Broken_Hill', 'Australia/Brisbane', 'Australia/Lindeman', 'Australia/Adelaide', 'Australia/Darwin', 'Australia/Perth'], 'code': 'AU', 'continent': 'Oceania', 'name': 'Australia', 'capital': 'Canberra'},
{'timezones': ['Asia/Baku'], 'code': 'AZ', 'continent': 'Asia', 'name': 'Azerbaijan', 'capital': 'Baku'},
{'timezones': ['America/Barbados'], 'code': 'BB', 'continent': 'north-america', 'name': 'Barbados', 'capital': 'Bridgetown'},
{'timezones': ['Asia/Dhaka'], 'code': 'BD', 'continent': 'Asia', 'name': 'Bangladesh', 'capital': 'Dhaka'},
{'timezones': ['Europe/Brussels'], 'code': 'BE', 'continent': 'Europe', 'name': 'Belgium', 'capital': 'Brussels'},
{'timezones': ['Africa/Ouagadougou'], 'code': 'BF', 'continent': 'Africa', 'name': 'Burkina Faso', 'capital': 'Ouagadougou'},
{'timezones': ['Europe/Sofia'], 'code': 'BG', 'continent': 'Europe', 'name': 'Bulgaria', 'capital': 'Sofia'},
{'timezones': ['Asia/Bahrain'], 'code': 'BH', 'continent': 'Asia', 'name': 'Bahrain', 'capital': 'Manama'},
{'timezones': ['Africa/Bujumbura'], 'code': 'BI', 'continent': 'Africa', 'name': 'Burundi', 'capital': 'Bujumbura'},
{'timezones': ['Africa/Porto-Novo'], 'code': 'BJ', 'continent': 'Africa', 'name': 'Benin', 'capital': 'Porto-Novo'},
{'timezones': ['Asia/Brunei'], 'code': 'BN', 'continent': 'Asia', 'name': 'Brunei Darussalam', 'capital': 'Bandar Seri Begawan'},
{'timezones': ['America/La_Paz'], 'code': 'BO', 'continent': 'south-america', 'name': 'Bolivia', 'capital': 'Sucre'},
{'timezones': ['America/Noronha', 'America/Belem', 'America/Fortaleza', 'America/Recife', 'America/Araguaina', 'America/Maceio', 'America/Bahia', 'America/Sao_Paulo', 'America/Campo_Grande', 'America/Cuiaba', 'America/Porto_Velho', 'America/Boa_Vista', 'America/Manaus', 'America/Eirunepe', 'America/Rio_Branco'], 'code': 'BR', 'continent': 'south-america', 'name': 'Brazil', 'capital': 'Bras\xc3\xadlia'},
{'timezones': ['America/Nassau'], 'code': 'BS', 'continent': 'north-america', 'name': 'Bahamas', 'capital': 'Nassau'},
{'timezones': ['Asia/Thimphu'], 'code': 'BT', 'continent': 'Asia', 'name': 'Bhutan', 'capital': 'Thimphu'},
{'timezones': ['Africa/Gaborone'], 'code': 'BW', 'continent': 'Africa', 'name': 'Botswana', 'capital': 'Gaborone'},
{'timezones': ['Europe/Minsk'], 'code': 'BY', 'continent': 'Europe', 'name': 'Belarus', 'capital': 'Minsk'},
{'timezones': ['America/Belize'], 'code': 'BZ', 'continent': 'north-america', 'name': 'Belize', 'capital': 'Belmopan'},
{'timezones': ['America/St_Johns', 'America/Halifax', 'America/Glace_Bay', 'America/Moncton', 'America/Goose_Bay', 'America/Blanc-Sablon', 'America/Montreal', 'America/Toronto', 'America/Nipigon', 'America/Thunder_Bay', 'America/Pangnirtung', 'America/Iqaluit', 'America/Atikokan', 'America/Rankin_Inlet', 'America/Winnipeg', 'America/Rainy_River', 'America/Cambridge_Bay', 'America/Regina', 'America/Swift_Current', 'America/Edmonton', 'America/Yellowknife', 'America/Inuvik', 'America/Dawson_Creek', 'America/Vancouver', 'America/Whitehorse', 'America/Dawson'], 'code': 'CA', 'continent': 'north-america', 'name': 'Canada', 'capital': 'Ottawa'},
{'timezones': ['Africa/Kinshasa', 'Africa/Lubumbashi'], 'code': 'CD', 'continent': 'Africa', 'name': 'Democratic Republic of the Congo', 'capital': 'Kinshasa'},
{'timezones': ['Africa/Brazzaville'], 'code': 'CG', 'continent': 'Africa', 'name': 'Republic of the Congo', 'capital': 'Brazzaville'},
{'timezones': ['Africa/Abidjan'], 'code': 'CI', 'continent': 'Africa', 'name': "C\xc3\xb4te d'Ivoire", 'capital': 'Yamoussoukro'},
{'timezones': ['America/Santiago', 'Pacific/Easter'], 'code': 'CL', 'continent': 'south-america', 'name': 'Chile', 'capital': 'Santiago'},
{'timezones': ['Africa/Douala'], 'code': 'CM', 'continent': 'Africa', 'name': 'Cameroon', 'capital': 'Yaound\xc3\xa9'},
{'timezones': ['Asia/Shanghai', 'Asia/Harbin', 'Asia/Chongqing', 'Asia/Urumqi', 'Asia/Kashgar'], 'code': 'CN', 'continent': 'Asia', 'name': "People's Republic of China", 'capital': 'Beijing'},
{'timezones': ['America/Bogota'], 'code': 'CO', 'continent': 'south-america', 'name': 'Colombia', 'capital': 'Bogot\xc3\xa1'},
{'timezones': ['America/Costa_Rica'], 'code': 'CR', 'continent': 'north-america', 'name': 'Costa Rica', 'capital': 'San Jos\xc3\xa9'},
{'timezones': ['America/Havana'], 'code': 'CU', 'continent': 'north-america', 'name': 'Cuba', 'capital': 'Havana'},
{'timezones': ['Atlantic/Cape_Verde'], 'code': 'CV', 'continent': 'Africa', 'name': 'Cape Verde', 'capital': 'Praia'},
{'timezones': ['Asia/Nicosia'], 'code': 'CY', 'continent': 'Asia', 'name': 'Cyprus', 'capital': 'Nicosia'},
{'timezones': ['Europe/Prague'], 'code': 'CZ', 'continent': 'Europe', 'name': 'Czech Republic', 'capital': 'Prague'},
{'timezones': ['Europe/Berlin'], 'code': 'DE', 'continent': 'Europe', 'name': 'Germany', 'capital': 'Berlin'},
{'timezones': ['Africa/Djibouti'], 'code': 'DJ', 'continent': 'Africa', 'name': 'Djibouti', 'capital': 'Djibouti City'},
{'timezones': ['Europe/Copenhagen'], 'code': 'DK', 'continent': 'Europe', 'name': 'Denmark', 'capital': 'Copenhagen'},
{'timezones': ['America/Dominica'], 'code': 'DM', 'continent': 'north-america', 'name': 'Dominica', 'capital': 'Roseau'},
{'timezones': ['America/Santo_Domingo'], 'code': 'DO', 'continent': 'north-america', 'name': 'Dominican Republic', 'capital': 'Santo Domingo'},
{'timezones': ['America/Guayaquil', 'Pacific/Galapagos'], 'code': 'EC', 'continent': 'south-america', 'name': 'Ecuador', 'capital': 'Quito'},
{'timezones': ['Europe/Tallinn'], 'code': 'EE', 'continent': 'Europe', 'name': 'Estonia', 'capital': 'Tallinn'},
{'timezones': ['Africa/Cairo'], 'code': 'EG', 'continent': 'Africa', 'name': 'Egypt', 'capital': 'Cairo'},
{'timezones': ['Africa/Asmera'], 'code': 'ER', 'continent': 'Africa', 'name': 'Eritrea', 'capital': 'Asmara'},
{'timezones': ['Africa/Addis_Ababa'], 'code': 'ET', 'continent': 'Africa', 'name': 'Ethiopia', 'capital': 'Addis Ababa'},
{'timezones': ['Europe/Helsinki'], 'code': 'FI', 'continent': 'Europe', 'name': 'Finland', 'capital': 'Helsinki'},
{'timezones': ['Pacific/Fiji'], 'code': 'FJ', 'continent': 'Oceania', 'name': 'Fiji', 'capital': 'Suva'},
{'timezones': ['Europe/Paris'], 'code': 'FR', 'continent': 'Europe', 'name': 'France', 'capital': 'Paris'},
{'timezones': ['Africa/Libreville'], 'code': 'GA', 'continent': 'Africa', 'name': 'Gabon', 'capital': 'Libreville'},
{'timezones': ['Asia/Tbilisi'], 'code': 'GE', 'continent': 'Asia', 'name': 'Georgia', 'capital': 'Tbilisi'},
{'timezones': ['Africa/Accra'], 'code': 'GH', 'continent': 'Africa', 'name': 'Ghana', 'capital': 'Accra'},
{'timezones': ['Africa/Banjul'], 'code': 'GM', 'continent': 'Africa', 'name': 'The Gambia', 'capital': 'Banjul'},
{'timezones': ['Africa/Conakry'], 'code': 'GN', 'continent': 'Africa', 'name': 'Guinea', 'capital': 'Conakry'},
{'timezones': ['Europe/Athens'], 'code': 'GR', 'continent': 'Europe', 'name': 'Greece', 'capital': 'Athens'},
{'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'north-america', 'name': 'Guatemala', 'capital': 'Guatemala City'},
{'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'north-america', 'name': 'Haiti', 'capital': 'Port-au-Prince'},
{'timezones': ['Africa/Bissau'], 'code': 'GW', 'continent': 'Africa', 'name': 'Guinea-Bissau', 'capital': 'Bissau'},
{'timezones': ['America/Guyana'], 'code': 'GY', 'continent': 'south-america', 'name': 'Guyana', 'capital': 'Georgetown'},
{'timezones': ['America/Tegucigalpa'], 'code': 'HN', 'continent': 'north-america', 'name': 'Honduras', 'capital': 'Tegucigalpa'},
{'timezones': ['Europe/Budapest'], 'code': 'HU', 'continent': 'Europe', 'name': 'Hungary', 'capital': 'Budapest'},
{'timezones': ['Asia/Jakarta', 'Asia/Pontianak', 'Asia/Makassar', 'Asia/Jayapura'], 'code': 'ID', 'continent': 'Asia', 'name': 'Indonesia', 'capital': 'Jakarta'},
{'timezones': ['Europe/Dublin'], 'code': 'IE', 'continent': 'Europe', 'name': 'Republic of Ireland', 'capital': 'Dublin'},
{'timezones': ['Asia/Jerusalem'], 'code': 'IL', 'continent': 'Asia', 'name': 'Israel', 'capital': 'Jerusalem'},
{'timezones': ['Asia/Calcutta'], 'code': 'IN', 'continent': 'Asia', 'name': 'India', 'capital': 'New Delhi'},
{'timezones': ['Asia/Baghdad'], 'code': 'IQ', 'continent': 'Asia', 'name': 'Iraq', 'capital': 'Baghdad'},
{'timezones': ['Asia/Tehran'], 'code': 'IR', 'continent': 'Asia', 'name': 'Iran', 'capital': 'Tehran'},
{'timezones': ['Atlantic/Reykjavik'], 'code': 'IS', 'continent': 'Europe', 'name': 'Iceland', 'capital': 'Reykjav\xc3\xadk'},
{'timezones': ['Europe/Rome'], 'code': 'IT', 'continent': 'Europe', 'name': 'Italy', 'capital': 'Rome'},
{'timezones': ['America/Jamaica'], 'code': 'JM', 'continent': 'north-america', 'name': 'Jamaica', 'capital': 'Kingston'},
{'timezones': ['Asia/Amman'], 'code': 'JO', 'continent': 'Asia', 'name': 'Jordan', 'capital': 'Amman'},
{'timezones': ['Asia/Tokyo'], 'code': 'JP', 'continent': 'Asia', 'name': 'Japan', 'capital': 'Tokyo'},
{'timezones': ['Africa/Nairobi'], 'code': 'KE', 'continent': 'Africa', 'name': 'Kenya', 'capital': 'Nairobi'},
{'timezones': ['Asia/Bishkek'], 'code': 'KG', 'continent': 'Asia', 'name': 'Kyrgyzstan', 'capital': 'Bishkek'},
{'timezones': ['Pacific/Tarawa', 'Pacific/Enderbury', 'Pacific/Kiritimati'], 'code': 'KI', 'continent': 'Oceania', 'name': 'Kiribati', 'capital': 'Tarawa'},
{'timezones': ['Asia/Pyongyang'], 'code': 'KP', 'continent': 'Asia', 'name': 'North Korea', 'capital': 'Pyongyang'},
{'timezones': ['Asia/Seoul'], 'code': 'KR', 'continent': 'Asia', 'name': 'South Korea', 'capital': 'Seoul'},
{'timezones': ['Asia/Kuwait'], 'code': 'KW', 'continent': 'Asia', 'name': 'Kuwait', 'capital': 'Kuwait City'},
{'timezones': ['Asia/Beirut'], 'code': 'LB', 'continent': 'Asia', 'name': 'Lebanon', 'capital': 'Beirut'},
{'timezones': ['Europe/Vaduz'], 'code': 'LI', 'continent': 'Europe', 'name': 'Liechtenstein', 'capital': 'Vaduz'},
{'timezones': ['Africa/Monrovia'], 'code': 'LR', 'continent': 'Africa', 'name': 'Liberia', 'capital': 'Monrovia'},
{'timezones': ['Africa/Maseru'], 'code': 'LS', 'continent': 'Africa', 'name': 'Lesotho', 'capital': 'Maseru'},
{'timezones': ['Europe/Vilnius'], 'code': 'LT', 'continent': 'Europe', 'name': 'Lithuania', 'capital': 'Vilnius'},
{'timezones': ['Europe/Luxembourg'], 'code': 'LU', 'continent': 'Europe', 'name': 'Luxembourg', 'capital': 'Luxembourg City'},
{'timezones': ['Europe/Riga'], 'code': 'LV', 'continent': 'Europe', 'name': 'Latvia', 'capital': 'Riga'},
{'timezones': ['Africa/Tripoli'], 'code': 'LY', 'continent': 'Africa', 'name': 'Libya', 'capital': 'Tripoli'},
{'timezones': ['Indian/Antananarivo'], 'code': 'MG', 'continent': 'Africa', 'name': 'Madagascar', 'capital': 'Antananarivo'},
{'timezones': ['Pacific/Majuro', 'Pacific/Kwajalein'], 'code': 'MH', 'continent': 'Oceania', 'name': 'Marshall Islands', 'capital': 'Majuro'},
{'timezones': ['Europe/Skopje'], 'code': 'MK', 'continent': 'Europe', 'name': 'Macedonia', 'capital': 'Skopje'},
{'timezones': ['Africa/Bamako'], 'code': 'ML', 'continent': 'Africa', 'name': 'Mali', 'capital': 'Bamako'},
{'timezones': ['Asia/Rangoon'], 'code': 'MM', 'continent': 'Asia', 'name': 'Myanmar', 'capital': 'Naypyidaw'},
{'timezones': ['Asia/Ulaanbaatar', 'Asia/Hovd', 'Asia/Choibalsan'], 'code': 'MN', 'continent': 'Asia', 'name': 'Mongolia', 'capital': 'Ulaanbaatar'},
{'timezones': ['Africa/Nouakchott'], 'code': 'MR', 'continent': 'Africa', 'name': 'Mauritania', 'capital': 'Nouakchott'},
{'timezones': ['Europe/Malta'], 'code': 'MT', 'continent': 'Europe', 'name': 'Malta', 'capital': 'Valletta'},
{'timezones': ['Indian/Mauritius'], 'code': 'MU', 'continent': 'Africa', 'name': 'Mauritius', 'capital': 'Port Louis'},
{'timezones': ['Indian/Maldives'], 'code': 'MV', 'continent': 'Asia', 'name': 'Maldives', 'capital': 'Mal\xc3\xa9'},
{'timezones': ['Africa/Blantyre'], 'code': 'MW', 'continent': 'Africa', 'name': 'Malawi', 'capital': 'Lilongwe'},
{'timezones': ['America/Mexico_City', 'America/Cancun', 'America/Merida', 'America/Monterrey', 'America/Mazatlan', 'America/Chihuahua', 'America/Hermosillo', 'America/Tijuana'], 'code': 'MX', 'continent': 'north-america', 'name': 'Mexico', 'capital': 'Mexico City'},
{'timezones': ['Asia/Kuala_Lumpur', 'Asia/Kuching'], 'code': 'MY', 'continent': 'Asia', 'name': 'Malaysia', 'capital': 'Kuala Lumpur'},
{'timezones': ['Africa/Maputo'], 'code': 'MZ', 'continent': 'Africa', 'name': 'Mozambique', 'capital': 'Maputo'},
{'timezones': ['Africa/Windhoek'], 'code': 'NA', 'continent': 'Africa', 'name': 'Namibia', 'capital': 'Windhoek'},
{'timezones': ['Africa/Niamey'], 'code': 'NE', 'continent': 'Africa', 'name': 'Niger', 'capital': 'Niamey'},
{'timezones': ['Africa/Lagos'], 'code': 'NG', 'continent': 'Africa', 'name': 'Nigeria', 'capital': 'Abuja'},
{'timezones': ['America/Managua'], 'code': 'NI', 'continent': 'north-america', 'name': 'Nicaragua', 'capital': 'Managua'},
{'timezones': ['Europe/Amsterdam'], 'code': 'NL', 'continent': 'Europe', 'name': 'Kingdom of the Netherlands', 'capital': 'Amsterdam'},
{'timezones': ['Europe/Oslo'], 'code': 'NO', 'continent': 'Europe', 'name': 'Norway', 'capital': 'Oslo'},
{'timezones': ['Asia/Katmandu'], 'code': 'NP', 'continent': 'Asia', 'name': 'Nepal', 'capital': 'Kathmandu'},
{'timezones': ['Pacific/Nauru'], 'code': 'NR', 'continent': 'Oceania', 'name': 'Nauru', 'capital': 'Yaren'},
{'timezones': ['Pacific/Auckland', 'Pacific/Chatham'], 'code': 'NZ', 'continent': 'Oceania', 'name': 'New Zealand', 'capital': 'Wellington'},
{'timezones': ['Asia/Muscat'], 'code': 'OM', 'continent': 'Asia', 'name': 'Oman', 'capital': 'Muscat'},
{'timezones': ['America/Panama'], 'code': 'PA', 'continent': 'north-america', 'name': 'Panama', 'capital': 'Panama City'},
{'timezones': ['America/Lima'], 'code': 'PE', 'continent': 'south-america', 'name': 'Peru', 'capital': 'Lima'},
{'timezones': ['Pacific/Port_Moresby'], 'code': 'PG', 'continent': 'Oceania', 'name': 'Papua New Guinea', 'capital': 'Port Moresby'},
{'timezones': ['Asia/Manila'], 'code': 'PH', 'continent': 'Asia', 'name': 'Philippines', 'capital': 'Manila'},
{'timezones': ['Asia/Karachi'], 'code': 'PK', 'continent': 'Asia', 'name': 'Pakistan', 'capital': 'Islamabad'},
{'timezones': ['Europe/Warsaw'], 'code': 'PL', 'continent': 'Europe', 'name': 'Poland', 'capital': 'Warsaw'},
{'timezones': ['Europe/Lisbon', 'Atlantic/Madeira', 'Atlantic/Azores'], 'code': 'PT', 'continent': 'Europe', 'name': 'Portugal', 'capital': 'Lisbon'},
{'timezones': ['Pacific/Palau'], 'code': 'PW', 'continent': 'Oceania', 'name': 'Palau', 'capital': 'Ngerulmud'},
{'timezones': ['America/Asuncion'], 'code': 'PY', 'continent': 'south-america', 'name': 'Paraguay', 'capital': 'Asunci\xc3\xb3n'},
{'timezones': ['Asia/Qatar'], 'code': 'QA', 'continent': 'Asia', 'name': 'Qatar', 'capital': 'Doha'},
{'timezones': ['Europe/Bucharest'], 'code': 'RO', 'continent': 'Europe', 'name': 'Romania', 'capital': 'Bucharest'},
{'timezones': ['Europe/Kaliningrad', 'Europe/Moscow', 'Europe/Volgograd', 'Europe/Samara', 'Asia/Yekaterinburg', 'Asia/Omsk', 'Asia/Novosibirsk', 'Asia/Krasnoyarsk', 'Asia/Irkutsk', 'Asia/Yakutsk', 'Asia/Vladivostok', 'Asia/Sakhalin', 'Asia/Magadan', 'Asia/Kamchatka', 'Asia/Anadyr'], 'code': 'RU', 'continent': 'Europe', 'name': 'Russia', 'capital': 'Moscow'},
{'timezones': ['Africa/Kigali'], 'code': 'RW', 'continent': 'Africa', 'name': 'Rwanda', 'capital': 'Kigali'},
{'timezones': ['Asia/Riyadh'], 'code': 'SA', 'continent': 'Asia', 'name': 'Saudi Arabia', 'capital': 'Riyadh'},
{'timezones': ['Pacific/Guadalcanal'], 'code': 'SB', 'continent': 'Oceania', 'name': 'Solomon Islands', 'capital': 'Honiara'},
{'timezones': ['Indian/Mahe'], 'code': 'SC', 'continent': 'Africa', 'name': 'Seychelles', 'capital': 'Victoria'},
{'timezones': ['Africa/Khartoum'], 'code': 'SD', 'continent': 'Africa', 'name': 'Sudan', 'capital': 'Khartoum'},
{'timezones': ['Europe/Stockholm'], 'code': 'SE', 'continent': 'Europe', 'name': 'Sweden', 'capital': 'Stockholm'},
{'timezones': ['Asia/Singapore'], 'code': 'SG', 'continent': 'Asia', 'name': 'Singapore', 'capital': 'Singapore'},
{'timezones': ['Europe/Ljubljana'], 'code': 'SI', 'continent': 'Europe', 'name': 'Slovenia', 'capital': 'Ljubljana'},
{'timezones': ['Europe/Bratislava'], 'code': 'SK', 'continent': 'Europe', 'name': 'Slovakia', 'capital': 'Bratislava'},
{'timezones': ['Africa/Freetown'], 'code': 'SL', 'continent': 'Africa', 'name': 'Sierra Leone', 'capital': 'Freetown'},
{'timezones': ['Europe/San_Marino'], 'code': 'SM', 'continent': 'Europe', 'name': 'San Marino', 'capital': 'San Marino'},
{'timezones': ['Africa/Dakar'], 'code': 'SN', 'continent': 'Africa', 'name': 'Senegal', 'capital': 'Dakar'},
{'timezones': ['Africa/Mogadishu'], 'code': 'SO', 'continent': 'Africa', 'name': 'Somalia', 'capital': 'Mogadishu'},
{'timezones': ['America/Paramaribo'], 'code': 'SR', 'continent': 'south-america', 'name': 'Suriname', 'capital': 'Paramaribo'},
{'timezones': ['Africa/Sao_Tome'], 'code': 'ST', 'continent': 'Africa', 'name': 'S\xc3\xa3o Tom\xc3\xa9 and Pr\xc3\xadncipe', 'capital': 'S\xc3\xa3o Tom\xc3\xa9'},
{'timezones': ['Asia/Damascus'], 'code': 'SY', 'continent': 'Asia', 'name': 'Syria', 'capital': 'Damascus'},
{'timezones': ['Africa/Lome'], 'code': 'TG', 'continent': 'Africa', 'name': 'Togo', 'capital': 'Lom\xc3\xa9'},
{'timezones': ['Asia/Bangkok'], 'code': 'TH', 'continent': 'Asia', 'name': 'Thailand', 'capital': 'Bangkok'},
{'timezones': ['Asia/Dushanbe'], 'code': 'TJ', 'continent': 'Asia', 'name': 'Tajikistan', 'capital': 'Dushanbe'},
{'timezones': ['Asia/Ashgabat'], 'code': 'TM', 'continent': 'Asia', 'name': 'Turkmenistan', 'capital': 'Ashgabat'},
{'timezones': ['Africa/Tunis'], 'code': 'TN', 'continent': 'Africa', 'name': 'Tunisia', 'capital': 'Tunis'},
{'timezones': ['Pacific/Tongatapu'], 'code': 'TO', 'continent': 'Oceania', 'name': 'Tonga', 'capital': 'Nuku\xca\xbbalofa'},
{'timezones': ['Europe/Istanbul'], 'code': 'TR', 'continent': 'Asia', 'name': 'Turkey', 'capital': 'Ankara'},
{'timezones': ['America/Port_of_Spain'], 'code': 'TT', 'continent': 'north-america', 'name': 'Trinidad and Tobago', 'capital': 'Port of Spain'},
{'timezones': ['Pacific/Funafuti'], 'code': 'TV', 'continent': 'Oceania', 'name': 'Tuvalu', 'capital': 'Funafuti'},
{'timezones': ['Africa/Dar_es_Salaam'], 'code': 'TZ', 'continent': 'Africa', 'name': 'Tanzania', 'capital': 'Dodoma'},
{'timezones': ['Europe/Kiev', 'Europe/Uzhgorod', 'Europe/Zaporozhye', 'Europe/Simferopol'], 'code': 'UA', 'continent': 'Europe', 'name': 'Ukraine', 'capital': 'Kiev'},
{'timezones': ['Africa/Kampala'], 'code': 'UG', 'continent': 'Africa', 'name': 'Uganda', 'capital': 'Kampala'},
{'timezones': ['America/New_York', 'America/Detroit', 'America/Kentucky/Louisville', 'America/Kentucky/Monticello', 'America/Indiana/Indianapolis', 'America/Indiana/Marengo', 'America/Indiana/Knox', 'America/Indiana/Vevay', 'America/Chicago', 'America/Indiana/Vincennes', 'America/Indiana/Petersburg', 'America/Menominee', 'America/North_Dakota/Center', 'America/North_Dakota/New_Salem', 'America/Denver', 'America/Boise', 'America/Shiprock', 'America/Phoenix', 'America/Los_Angeles', 'America/Anchorage', 'America/Juneau', 'America/Yakutat', 'America/Nome', 'America/Adak', 'Pacific/Honolulu'], 'code': 'US', 'continent': 'north-america', 'name': 'United States', 'capital': 'Washington, D.C.'},
{'timezones': ['America/Montevideo'], 'code': 'UY', 'continent': 'south-america', 'name': 'Uruguay', 'capital': 'Montevideo'},
{'timezones': ['Asia/Samarkand', 'Asia/Tashkent'], 'code': 'UZ', 'continent': 'Asia', 'name': 'Uzbekistan', 'capital': 'Tashkent'},
{'timezones': ['Europe/Vatican'], 'code': 'VA', 'continent': 'Europe', 'name': 'Vatican City', 'capital': 'Vatican City'},
{'timezones': ['America/Caracas'], 'code': 'VE', 'continent': 'south-america', 'name': 'Venezuela', 'capital': 'Caracas'},
{'timezones': ['Asia/Saigon'], 'code': 'VN', 'continent': 'Asia', 'name': 'Vietnam', 'capital': 'Hanoi'},
{'timezones': ['Pacific/Efate'], 'code': 'VU', 'continent': 'Oceania', 'name': 'Vanuatu', 'capital': 'Port Vila'},
{'timezones': ['Asia/Aden'], 'code': 'YE', 'continent': 'Asia', 'name': 'Yemen', 'capital': "Sana'a"},
{'timezones': ['Africa/Lusaka'], 'code': 'ZM', 'continent': 'Africa', 'name': 'Zambia', 'capital': 'Lusaka'},
{'timezones': ['Africa/Harare'], 'code': 'ZW', 'continent': 'Africa', 'name': 'Zimbabwe', 'capital': 'Harare'},
{'timezones': ['Africa/Algiers'], 'code': 'DZ', 'continent': 'Africa', 'name': 'Algeria', 'capital': 'Algiers'},
{'timezones': ['Europe/Sarajevo'], 'code': 'BA', 'continent': 'Europe', 'name': 'Bosnia and Herzegovina', 'capital': 'Sarajevo'},
{'timezones': ['Asia/Phnom_Penh'], 'code': 'KH', 'continent': 'Asia', 'name': 'Cambodia', 'capital': 'Phnom Penh'},
{'timezones': ['Africa/Bangui'], 'code': 'CF', 'continent': 'Africa', 'name': 'Central African Republic', 'capital': 'Bangui'},
{'timezones': ['Africa/Ndjamena'], 'code': 'TD', 'continent': 'Africa', 'name': 'Chad', 'capital': "N'Djamena"},
{'timezones': ['Indian/Comoro'], 'code': 'KM', 'continent': 'Africa', 'name': 'Comoros', 'capital': 'Moroni'},
{'timezones': ['Europe/Zagreb'], 'code': 'HR', 'continent': 'Europe', 'name': 'Croatia', 'capital': 'Zagreb'},
{'timezones': ['Asia/Dili'], 'code': 'TL', 'continent': 'Asia', 'name': 'East Timor', 'capital': 'Dili'},
{'timezones': ['America/El_Salvador'], 'code': 'SV', 'continent': 'north-america', 'name': 'El Salvador', 'capital': 'San Salvador'},
{'timezones': ['Africa/Malabo'], 'code': 'GQ', 'continent': 'Africa', 'name': 'Equatorial Guinea', 'capital': 'Malabo'},
{'timezones': ['America/Grenada'], 'code': 'GD', 'continent': 'north-america', 'name': 'Grenada', 'capital': "St. George's"},
{'timezones': ['Asia/Almaty', 'Asia/Qyzylorda', 'Asia/Aqtobe', 'Asia/Aqtau', 'Asia/Oral'], 'code': 'KZ', 'continent': 'Asia', 'name': 'Kazakhstan', 'capital': 'Astana'},
{'timezones': ['Asia/Vientiane'], 'code': 'LA', 'continent': 'Asia', 'name': 'Laos', 'capital': 'Vientiane'},
{'timezones': ['Pacific/Truk', 'Pacific/Ponape', 'Pacific/Kosrae'], 'code': 'FM', 'continent': 'Oceania', 'name': 'Federated States of Micronesia', 'capital': 'Palikir'},
{'timezones': ['Europe/Chisinau'], 'code': 'MD', 'continent': 'Europe', 'name': 'Moldova', 'capital': 'Chi\xc5\x9fin\xc4\x83u'},
{'timezones': ['Europe/Monaco'], 'code': 'MC', 'continent': 'Europe', 'name': 'Monaco', 'capital': 'Monaco'},
{'timezones': ['Europe/Podgorica'], 'code': 'ME', 'continent': 'Europe', 'name': 'Montenegro', 'capital': 'Podgorica'},
{'timezones': ['Africa/Casablanca'], 'code': 'MA', 'continent': 'Africa', 'name': 'Morocco', 'capital': 'Rabat'},
{'timezones': ['America/St_Kitts'], 'code': 'KN', 'continent': 'north-america', 'name': 'Saint Kitts and Nevis', 'capital': 'Basseterre'},
{'timezones': ['America/St_Lucia'], 'code': 'LC', 'continent': 'north-america', 'name': 'Saint Lucia', 'capital': 'Castries'},
{'timezones': ['America/St_Vincent'], 'code': 'VC', 'continent': 'north-america', 'name': 'Saint Vincent and the Grenadines', 'capital': 'Kingstown'},
{'timezones': ['Pacific/Apia'], 'code': 'WS', 'continent': 'Oceania', 'name': 'Samoa', 'capital': 'Apia'},
{'timezones': ['Europe/Belgrade'], 'code': 'RS', 'continent': 'Europe', 'name': 'Serbia', 'capital': 'Belgrade'},
{'timezones': ['Africa/Johannesburg'], 'code': 'ZA', 'continent': 'Africa', 'name': 'South Africa', 'capital': 'Pretoria'},
{'timezones': ['Europe/Madrid', 'Africa/Ceuta', 'Atlantic/Canary'], 'code': 'ES', 'continent': 'Europe', 'name': 'Spain', 'capital': 'Madrid'},
{'timezones': ['Asia/Colombo'], 'code': 'LK', 'continent': 'Asia', 'name': 'Sri Lanka', 'capital': 'Sri Jayewardenepura Kotte'},
{'timezones': ['Africa/Mbabane'], 'code': 'SZ', 'continent': 'Africa', 'name': 'Swaziland', 'capital': 'Mbabane'},
{'timezones': ['Europe/Zurich'], 'code': 'CH', 'continent': 'Europe', 'name': 'Switzerland', 'capital': 'Bern'},
{'timezones': ['Asia/Dubai'], 'code': 'AE', 'continent': 'Asia', 'name': 'United Arab Emirates', 'capital': 'Abu Dhabi'},
{'timezones': ['Europe/London'], 'code': 'GB', 'continent': 'Europe', 'name': 'United Kingdom', 'capital': 'London'}
]

In [134]:
#Import Data
df = pd.read_json('data_lonelyPlanet.json')
df_global = pd.DataFrame(countries)
df_world = pd.read_json('world-cities_json.json')

#clean the two new dataframes:
df_global = df_global.drop(['timezones','capital','code'], axis=1)
df_global['continent'] = df_global['continent'].str.lower()
df_global['name'] = df_global['name'].str.lower()
df_global = df_global.rename(index=str, columns={'name':'country'})

#clean the two new dataframes:
df_world = df_world.drop(['geonameid','subcountry'], axis=1)
df_world['country'] = df_world['country'].str.lower()
df_world['name'] = df_world['name'].str.lower()
df_world = df_world.rename(index=str, columns={'name':'town'})

In [135]:
len(df.title)

3400

In [136]:
#Lets clean up the Title column
df['title'] = df['title'].apply(lambda x: x.replace('\n ', ''))
df['title'] = df['title'].apply(lambda x: x.strip(''))

#And lets get an additional column for doc2vec later on
df['title_article'] = df['title']

In [137]:
def getCountry(x):
    theString = x[29:]
    
    if theString.split('/')[0] == 'travel-tips-and-articles':
        return 'none'
    else:
        return theString.split('/')[0]
    
def getTown(x):
    theString = x[29:]
    
    if theString.split('/')[1] == 'travel-tips-and-articles':
        return 'none'
    else:
        return theString.split('/')[1]

df['country'] = list(map(getCountry, df['URL']))
df['town'] = list(map(getTown, df['URL']))

In [138]:
townDict = dict(zip(df_world.town, df_world.country))
townList = df_world.town.unique().tolist()
countryList = df_world.country.unique().tolist()
countryDict = dict(zip(df_global.country, df_global.continent))
continentList = df_global.continent.unique().tolist()

In [139]:

def townClean(article, country, town):
    if(country == 'none') or (country != 'none' and town == 'none'):
        wordList = wordpunct_tokenize(article.lower())
        returnWord = 'none'
        for word in wordList:
            word = word.replace(' ','')
            if word in townList:
                returnWord = word
    
        if returnWord == 'best' or returnWord == 'of' or returnWord == 'most' or returnWord == 'paradise':
            return 'none'
        else:
            return returnWord
    else:
        return town
        

    
def countryClean(article, country, town):
    
    wordList = wordpunct_tokenize(article.lower())
    returnWord = 'none'
    
    if(country == 'none' and town != 'none'):
        if town in townDict:
            returnWord = townDict[town]
    elif(country == 'none' and town == 'none'):
        for word in wordList:
            word = word.replace(' ','')
            if word in countryList:
                returnWord = word
    elif(country != 'none'):
        if country == 'north-america' or country == 'south-america' or country == 'antarctica' or country == 'europe' or country == 'asia' or country == 'africa':
            returnWord = 'none'
        else:
            returnWord = country
    
    return returnWord
        
def continentClean(article, country):
    
    wordList = wordpunct_tokenize(article.lower())
    found = 0
    returnWord = 'none'
    
    if country != 'none':
        if country in countryDict:
            returnWord = countryDict[country]
        elif country == 'china' or country == 'south-korea' or country == 'east-timor' or country == 'taiwan' or country == 'central-asia' or country == 'southeast-asia' or country == 'asia' or country == 'borneo' or country == 'myanmar-burma' or country == 'sri-lanka' or country == 'north-korea' or country == 'timor-leste' or country == 'brunei-darussalam':
            returnWord = 'asia'
        elif country == 'netherlands' or country == 'wales' or country == 'ireland' or country == 'england' or country == 'europe' or country == 'the-netherlands' or country == 'czech-republic' or country == 'mediterranean-europe' or country == 'western-europe' or country == 'eastern-europe' or country == 'the-united-kingdom' or country == 'scotland' or country == 'scandinavia' or country == 'bosnia-and-hercegovina' or country == 'great-britain' or country == 'vatican-city' or country == 'central-europe' or country == 'canary-islands' or country == 'kosovo' or country == 'faroe-islands':
            returnWord = 'europe'
        elif country == 'israel-and-the-palestinian-territories' or country == 'united-arab-emirates' or country == 'arabian-peninsula-1333684' or country == 'saudi-arabia':
            returnWord = 'middle-east'
        elif country == 'new-zealand ' or country == ' new-zealand' or country == 'new-zealand' or country =='south-pacific' or country == 'pacific' or country == 'solomon-islands' or country == 'tahiti-and-french-polynesia' or country == 'rarotonga-and-the-cook-islands' or country == 'niue' or country == 'papua-new-guinea' or country == 'new-caledonia':
            returnWord = 'oceania'
        elif country == 'usa' or country == 'north-america' or country == 'the-pacific-northwest' or country == 'caribbean' or country == 'puerto-rico' or country == 'the-bahamas' or country == 'americas' or country == 'dominican-republic' or country == 'costa-rica' or country == 'bermuda' or country == 'central-america' or country == 'cayman-islands' or country == 'greenland' or country == 'st-lucia' or country == 'us-virgin-islands' or country == 'guadeloupe' or country == 'aruba-bonaire-and-curacao' or country == 'windward-islands' or country == 'martinique' or country == 'st-martin-sint-maarten-1336613' or country == 'turks-and-caicos':
            returnWord = 'north-america'
        elif country == 'south-georgia' or country == 'antarctic-peninsula' or country == 'antarctica-1007062':
            return 'antartica'
        elif country == 'south-america' or country == 'el-salvador' or country == 'st-vincent-and-the-grenadines' or country == 'the-guianas':
            returnWord = 'south-america'
        elif country == 'south-africa' or country == 'democratic-republic-of-congo' or country == 'africa' or country == 'west-africa' or country == 'congo' or country == 'central-african-republic' or country == 'ivory coast' or country == 'the-gambia' or country == 'south-africa-lesotho-swaziland':
            returnWord = 'africa'
    
    else:
        if wordList[0] in continentList:
            returnWord == wordList[0]
        else:
            for i in range(1, len(wordList)):
                temp = wordList[i - 1] + " " + wordList[i]
                if (temp == 'south_america' or 'latin_america') and found == 0:
                    returnWord = 'south america'
                    found += 1
                elif temp == 'north_america' and found == 0:
                    returnWord = 'north america'
                    found += 1
                elif wordList[i] in continentList:
                    returnWord = wordList[i]
                else:
                    returnWord = 'none'
            
    return returnWord

In [140]:
df['town'] = list(map(townClean, df['title_article'], df['country'], df['town']))
df['country'] = list(map(countryClean, df['title_article'], df['country'], df['town']))
df['continent'] = list(map(continentClean, df['title_article'], df['country']))

In [141]:
townList = df['town'].unique().tolist()
countryList = df['country'].unique().tolist()
doubleTowns = []
doubleCountry = []

for i in townList:
    if (len(i.split('-')) == 2):
        doubleTowns.append(i)
        
for j in countryList:
    if (len(j.split('-')) == 2):
        doubleCountry.append(j)


doubleCountry


['the-netherlands',
 'czech-republic',
 'south-korea',
 'el-salvador',
 'saudi-arabia',
 'western-europe',
 'new-zealand',
 'puerto-rico',
 'central-europe',
 'dominican-republic',
 'southeast-asia',
 'great-britain',
 'south-africa',
 'windward-islands',
 'the-bahamas',
 'sri-lanka',
 'eastern-europe',
 'the-gambia',
 'antarctica-1007062',
 'central-asia',
 'canary-islands',
 'myanmar-burma',
 'costa-rica',
 'antarctic-peninsula',
 'new-caledonia',
 'st-lucia',
 'east-timor',
 'vatican-city',
 'north-korea',
 'timor-leste',
 'brunei-darussalam',
 'faroe-islands',
 'cayman-islands',
 'west-africa',
 'mediterranean-europe',
 'central-america',
 'the-guianas',
 'solomon-islands',
 'south-georgia',
 'south-pacific']

In [142]:
df[df['country'] == 'brazil']

,URL,article,images,title,title_article,country,town,continent
214,https://www.lonelyplanet.com/brazil/rio-de-jan...,[Long-distance walking trails inspire a fascin...,[https://lonelyplanetwp.imgix.net/2017/10/Tril...,Trekking Rio on the Transcarioca Tr...,Trekking Rio on the Transcarioca Tr...,brazil,rio-de-janeiro,south-america
346,https://www.lonelyplanet.com/brazil/bahia/trav...,"[But a glance at <a href=""https://www.lonelypl...",[https://lonelyplanetwp.imgix.net/2017/08/1.-w...,Best of Bahia state: Brazil's 'land...,Best of Bahia state: Brazil's 'land...,brazil,bahia,south-america
925,https://www.lonelyplanet.com/travel-tips-and-a...,[If budget travel makes you think of noisy dor...,[https://lonelyplanetwp.imgix.net/2016/08/Luca...,"Meet a traveller: Lucas Peterson, b...","Meet a traveller: Lucas Peterson, b...",brazil,lucas,south-america
962,https://www.lonelyplanet.com/brazil/rio-de-jan...,"[Quite a lot, as a matter of fact. <a href=""ht...",[https://lonelyplanetwp.imgix.net/2012/12/esca...,Ten beach-free things to do in Rio ...,Ten beach-free things to do in Rio ...,brazil,rio-de-janeiro,south-america
974,https://www.lonelyplanet.com/brazil/rio-de-jan...,[Rio de Janeiro's markets (known in Portuguese...,[https://lonelyplanetwp.imgix.net/2016/07/Feir...,From caipirinhas&nbsp;to carne seca...,From caipirinhas&nbsp;to carne seca...,brazil,rio-de-janeiro,south-america
1005,https://www.lonelyplanet.com/brazil/rio-de-jan...,[Food is a treasured part of <em>carioca</em> ...,[https://lonelyplanetwp.imgix.net/2015/02/Chur...,Eat like a carioca: Rio's best loca...,Eat like a carioca: Rio's best loca...,brazil,rio-de-janeiro,south-america
1170,https://www.lonelyplanet.com/brazil/travel-tip...,[The world's fifth-largest country commandeers...,[https://lonelyplanetwp.imgix.net/2013/12/carn...,A beginner's guide to Brazil,A beginner's guide to Brazil,brazil,none,south-america
1344,https://www.lonelyplanet.com/travel-tips-and-a...,"[ \n <a href=""http://www.lonelyplanet.com/trav...",[https://lonelyplanetwp.imgix.net/2015/09/Ilha...,Ilha Grande: tropical island paradi...,Ilha Grande: tropical island paradi...,brazil,none,south-america
1607,https://www.lonelyplanet.com/brazil/rio-de-jan...,[Rio takes an appropriately laid back approach...,[https://lonelyplanetwp.imgix.net/2015/04/DSCF...,A night out in Rio,A night out in Rio,brazil,rio-de-janeiro,south-america
1647,https://www.lonelyplanet.com/brazil/rio-de-jan...,[The samba parade is carnival’s main event and...,[https://lonelyplanetwp.imgix.net/2015/04/IMG_...,On the ground at Rio’s Carnaval: a ...,On the ground at Rio’s Carnaval: a ...,brazil,rio-de-janeiro,south-america


In [143]:
articleClean_raw = []
articleTitle_raw = []

article_List = df['article'].tolist()

for article in article_List:
    theText = []
    for i in article:
        clean = re.sub('<[^>]+>', '', i)
        clean = clean.replace(' \n', '')
        clean = clean.replace('  ',' ')
        theText.append(clean) 
    articleClean_raw.append(theText)

In [144]:
docLabels_raw = df['title'].tolist()
docCountry_raw = df['country'].tolist()
docTown_raw = df['town'].tolist()
docContinent_raw = df['continent'].tolist()
article_TitleList_raw = df['title_article'].tolist()


articleClean = []
docLabels = []
docLabelsTitle = []


#use the for loops to build the article labels that will help up later
for i in range(len(articleClean_raw)):
    article = articleClean_raw[i]
    for par in range(len(article)):
        articleClean.append(article[par])
        if par < 10:
            docLab = docLabels_raw[i].strip() + '0' + str(par) + '_' + docCountry_raw[i] + '_' + docTown_raw[i] + '_' + docContinent_raw[i]
        else:
            docLab = docLabels_raw[i].strip() + str(par) + '_' + docCountry_raw[i] + '_' + docTown_raw[i] + '_' + docContinent_raw[i]
        docLabels.append(docLab)

        
#Now lets do the same process for the article titles
for i in range(len(article_TitleList_raw)):
    docLab = article_TitleList_raw[i].strip() + '00_' + docCountry_raw[i] + '_' + docTown_raw[i] + '_' + docContinent_raw[i]
    docLabelsTitle.append(docLab)
    


In [145]:
#Get a zip of article titles, image, and URL

def getFirstImage(x):
    
    if len(x) > 1:
        return x[0]
    elif len(x) == 1:
        return 'https://fthmb.tqn.com/NQWHLZNDNQ-z6d2bFjOhEa7_jmo=/960x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/475689807-56a8d2f95f9b58b7d0f57954.jpg'
    else:
        return 'http://static1.squarespace.com/static/5779ddf6d1758e240176dffd/589d0b1d20099e45235145e4/589d0bf2bf629a9c5e66adca/1519679096230/iStock-522001766.jpg?format=1000w'

df['title_article'] = df['title_article'].apply(lambda x: x.strip())
df['image_one'] = list(map(getFirstImage, df['images']))

df_infoLink = df[['title_article','image_one','URL']]
df_infoLink = df_infoLink.rename(index=str, columns={'title_article':'title'})

# Lets build the Doc2Vec for the paragraphs in the articles

In [146]:
tokenizer = RegexpTokenizer(r"\w+")
stopword_set = set(stopwords.words('english'))

def nlp_clean(data):
    
    new_data = []
    
    for d in data:
        new_str = d.lower()
        dlist = tokenizer.tokenize(new_str)
        dlist = list(set(dlist).difference(stopword_set))
        new_data.append(dlist)
        
    return new_data

#Lets run it for the paragraphs in the articles
finalData = nlp_clean(articleClean)

#Lets run it for article title
finalData_title = nlp_clean(article_TitleList_raw)

In [147]:
finalData

[['famous',
  'backs',
  'global',
  'first',
  'british',
  'left',
  'status',
  'religious',
  'turned',
  'monumental',
  'people',
  'citizens',
  'age',
  'fearsome',
  'wealthy',
  'architecture',
  'construction',
  'towering',
  'less',
  'nation',
  'mosques',
  'world',
  'would',
  'embraced',
  'temples',
  'india',
  'tradition',
  'historical',
  'known',
  'years',
  'well',
  'new',
  'behind',
  'hybrid',
  'second',
  'fortresses',
  'explosion',
  'buildings',
  'raj',
  'wars',
  'underwent',
  'curious',
  'indians',
  'file',
  'comes'],
 ['miami',
  'deco',
  'lined',
  'mumbai',
  'images',
  'iconic',
  'getty',
  'art',
  'paranjpe',
  'apartments',
  'seafront',
  'punit',
  'like'],
 ['runaway',
  'north',
  'playground',
  'embodied',
  'school',
  'iconography',
  'iconic',
  'architects',
  'built',
  'old',
  'cinema',
  'inspired',
  'comic',
  'reclaimed',
  'apartment',
  'modern',
  'décoratifs',
  'quarters',
  'city',
  'internationale',
  'flash'

In [148]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.TaggedDocument(doc,    
[self.labels_list[idx]])

In [149]:
it = LabeledLineSentence(finalData, docLabels)
it_t = LabeledLineSentence(finalData_title, docLabelsTitle)

In [150]:
#Fire up Doc2Vec for article paragraphs
abmodel = gensim.models.doc2vec.Doc2Vec(size=300, window=8, min_count=3, alpha = .025, min_alpha = .025, workers=40)
abmodel.build_vocab(it)
print('abstract model built')
a = len(finalData)
for epoch in range(10):
    abmodel.train(it, total_examples = a, epochs = 1)
    abmodel.alpha -= 0.002 # decrease the learning rate
    abmodel.min_alpha = abmodel.alpha # fix the learning rate, no deca
    abmodel.train(it, total_examples = a, epochs = 1)

/Users/bradleydavies/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


abstract model built


In [151]:
#Fire up Doc2Vec for article titles
abmodel_t = gensim.models.doc2vec.Doc2Vec(size=300, window=8, min_count=3, alpha = .025, min_alpha = .025, workers=40)
abmodel_t.build_vocab(it_t)
print('abstract model built')
a = len(finalData_title)
for epoch in range(10):
    abmodel_t.train(it_t, total_examples = a, epochs = 1)
    abmodel_t.alpha -= 0.002 # decrease the learning rate
    abmodel_t.min_alpha = abmodel_t.alpha # fix the learning rate, no deca
    abmodel_t.train(it_t, total_examples = a, epochs = 1)

/Users/bradleydavies/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


abstract model built


# Pickle everything we need for Flask

In [152]:
#Doc2Vec models
pickle.dump(abmodel, open("abmodel.pkl", "wb"))
pickle.dump(abmodel_t, open("abmodel_t.pkl", "wb"))

pickle.dump(doubleTowns, open("doubleTowns.pkl", "wb"))
pickle.dump(doubleCountry, open("doubleCountry.pkl", "wb"))

pickle.dump(df_infoLink, open("df_infoLink.pkl", "wb"))

# Now Lets build the Doc2Vec for the Article Title

In [153]:
search = "I want a food journey in Spain"

In [154]:
#lets check for major 2 word key phrases by using the doubleTowns, doubleCountry, and check for north and south america
query = search.lower().strip("!.?").replace(',','')
query = query.split(' ')
q_temp = query
rQuery  = [q_temp[0]]

for i in range(1, len(q_temp)):
    if (q_temp[i-1] + '-' + q_temp[i]) in doubleTowns:
        rQuery.pop()
        rQuery.append(q_temp[i-1] + '-' + q_temp[i])
    elif (q_temp[i-1] + '-' + q_temp[i]) in doubleCountry:
        rQuery.pop()
        rQuery.append(q_temp[i-1] + '-' + q_temp[i])
    elif (q_temp[i-1] + '-' + q_temp[i]) == 'south-america' or (q_temp[i-1] + '-' + q_temp[i]) == 'north-america' or (q_temp[i-1] + '-' + q_temp[i]) == 'middle-east':
        rQuery.pop()
        rQuery.append(q_temp[i-1] + '-' + q_temp[i])
    else:
        rQuery.append(q_temp[i])
        
query = ' '.join(word for word in rQuery)

In [155]:
query

'i want a food journey in spain'

In [156]:
query = word_tokenize(query)
query = list(set(query).difference(stopword_set))

q_vec = abmodel.infer_vector(query)
results = abmodel.docvecs.most_similar(positive=[q_vec], topn = 100000)

q_vec_t = abmodel_t.infer_vector(query)
results_t = abmodel_t.docvecs.most_similar(positive=[q_vec_t], topn = 100000)

In [157]:
query

['spain', 'want', 'food', 'journey']

In [158]:
df_results = pd.DataFrame(results, columns=['title','similarity'])
df_results_t = pd.DataFrame(results_t, columns=['title','similarity'])

In [159]:
def isCountry(x):
    
    country = x.split('_')[1]
    theQ = query
    
    if country in theQ:
        return 1
    else:
        return 0
    
def isTown(x):
    
    town = x.split('_')[2]
    theQ = query
    
    if town in theQ:
        return 1
    else:
        return 0
    
def isContinent(x):
    
    continent = x.split('_')[3]
    theQ = query
    
    if continent in theQ:
        return 1
    else:
        return 0
    
    

#Break up the title piece
df_results['firstParagraph'] = df_results['title'].apply(lambda x: 1 if x.split('_')[0][-2:] == '00' else 0)
df_results['isCountry'] = list(map(isCountry, df_results['title']))
df_results['isTown'] = list(map(isTown, df_results['title']))
df_results['isContinent'] = list(map(isContinent, df_results['title']))

df_results_t['firstParagraph'] = df_results_t['title'].apply(lambda x: 1 if x.split('_')[0][-2:] == '00' else 0)
df_results_t['isCountry'] = list(map(isCountry, df_results_t['title']))
df_results_t['isTown'] = list(map(isTown, df_results_t['title']))
df_results_t['isContinent'] = list(map(isContinent, df_results_t['title']))

#reset title column and sort values to get the results sorted
df_results['title'] = df_results['title'].apply(lambda x: x.split('_')[0][:-2])
df_results_t['title'] = df_results_t['title'].apply(lambda x: x.split('_')[0][:-2])

#Filter for results containing a city or country
def getResults(df):
    df_temp1 = df[df['isTown'] == 1]
    if len(df_temp1) == 0:
        df_temp2 = df[df['isCountry'] == 1]
        if len(df_temp2) == 0:
            df_temp3 = df[df['isContinent'] == 1]
            if len(df_temp3) == 0:
                return df
            else:
                return df_temp3
        else:
            return df_temp2
    else:
        return df_temp1
    
df_results = getResults(df_results)
df_results_t = getResults(df_results_t)

#Break up between first paragraph similarities and non-first
df_results_firstP = df_results[df_results['firstParagraph'] == 1]
df_results = df_results[df_results['firstParagraph'] == 0]

#Get the top similarity per article 
df_results = df_results.sort_values(['title', 'similarity'], ascending=[True, False])
df_results_big = df_results.drop_duplicates('title', keep='first')

In [160]:
df_results.head(10)

,title,similarity,firstParagraph,isCountry,isTown,isContinent
74,10 places to try old-school Spanish food in Ma...,0.545698,0,1,0,0
14768,10 places to try old-school Spanish food in Ma...,-0.080034,0,1,0,0
19781,10 places to try old-school Spanish food in Ma...,-0.218367,0,1,0,0
24253,10 places to try old-school Spanish food in Ma...,-0.302449,0,1,0,0
24766,10 places to try old-school Spanish food in Ma...,-0.310155,0,1,0,0
25377,10 places to try old-school Spanish food in Ma...,-0.319304,0,1,0,0
33472,10 places to try old-school Spanish food in Ma...,-0.394234,0,1,0,0
33561,10 places to try old-school Spanish food in Ma...,-0.394863,0,1,0,0
35867,10 places to try old-school Spanish food in Ma...,-0.409368,0,1,0,0
42237,10 places to try old-school Spanish food in Ma...,-0.439967,0,1,0,0


In [161]:
def makeCombo(x, y):
    return str(x) + '_' + str(y)

df_results['combo'] = list(map(makeCombo, df_results['title'], df_results['similarity']))
df_results_big['combo'] = list(map(makeCombo, df_results_big['title'], df_results_big['similarity']))

naughty = df_results_big['combo'].tolist()

def makeTable(x):
    
    
    if x in naughty:
        return 1
    else:
        return 0
    
df_results['combo_link'] = list(map(makeTable, df_results['combo']))

df_results = df_results[df_results['combo_link'] == 0]

/Users/bradleydavies/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [162]:
#Body article scores
body = df_results.groupby('title').mean().sort_values(by='similarity', ascending=False).reset_index()
body = body.drop(['firstParagraph','isCountry','isTown','isContinent','combo_link'], axis=1)
body = body.rename(index=str, columns={'similarity':'similarity_body'})
body['similarity_body'] = body['similarity_body'].apply(lambda x: x*0.27)

body

,title,similarity_body
0,Top 10 architectural gems of Barcelona,0.022471
1,On the tapas trail in Seville,-0.047005
2,Unexplored Spain: road-tripping around Extrema...,-0.048927
3,Barcelona on film,-0.066975
4,Madrid and the art of tapas,-0.067225
5,"Ten ways to break a sweat in Mallorca, Spain",-0.069242
6,Walking the camino: our tips,-0.074507
7,A guide to the eateries of Spain,-0.075140
8,Andalucía's local hangout: Costa de la Luz,-0.075530
9,Spain's flamenco renaissance,-0.075980


In [163]:
#First Paragraph scores
firstP = df_results_firstP.groupby('title').mean().sort_values(by='similarity', ascending=False).reset_index()
firstP = firstP.drop(['firstParagraph','isCountry','isTown','isContinent'], axis=1)
firstP = firstP.rename(index=str, columns={'similarity':'similarity_firstP'})
firstP['similarity_firstP'] = firstP['similarity_firstP'].apply(lambda x: x*0.70)

In [164]:
#Title scores
title = df_results_t.groupby('title').mean().sort_values(by='similarity', ascending=False).reset_index()
title = title.drop(['firstParagraph','isCountry','isTown','isContinent'], axis=1)
title = title.rename(index=str, columns={'similarity':'similarity_title'})
title['similarity_title'] = title['similarity_title'].apply(lambda x: x*0.03)

In [165]:
final_temp = pd.merge(firstP, body, how='left', on='title')
df_final = pd.merge(final_temp, title, how='left', on='title')

def getFinalScore(x,y,z):
    return x+y+z

df_final['final_score'] = list(map(getFinalScore, df_final['similarity_body'],df_final['similarity_firstP'], df_final['similarity_title']))
df_final = df_final.sort_values(by='final_score', ascending=False)

In [166]:
df_finalFinal = pd.merge(df_final, df_infoLink, how='left', on='title')
df_finalFinal

,title,similarity_firstP,similarity_body,similarity_title,final_score,image_one,URL
0,Madrid and the art of tapas,0.321000,-0.067225,-0.006060,0.247715,https://fthmb.tqn.com/NQWHLZNDNQ-z6d2bFjOhEa7_...,https://www.lonelyplanet.com/spain/madrid/trav...
1,Andalucía's local hangout: Costa de la Luz,0.281393,-0.075530,-0.005939,0.199924,http://static1.squarespace.com/static/5779ddf6...,https://www.lonelyplanet.com/spain/andalucia/h...
2,An introduction to Catalan cuisine,0.296120,-0.109528,-0.006125,0.180467,https://lonelyplanetwp.imgix.net/2015/02/Jos-i...,https://www.lonelyplanet.com/spain/barcelona/t...
3,Galicia: exploring the crossroads of Spanish a...,0.278386,-0.096894,-0.006122,0.175370,https://lonelyplanetwp.imgix.net/2017/03/galic...,https://www.lonelyplanet.com/spain/galicia/tra...
4,Architectural sights of Andalucía,0.259787,-0.132101,-0.005975,0.121712,http://inside-digital.blog.lonelyplanet.com/wo...,https://www.lonelyplanet.com/spain/andalucia/t...
5,24 food-filled hours in San Sebastián,0.163708,-0.104124,0.005344,0.064927,https://lonelyplanetwp.imgix.net/2016/01/Bokka...,https://www.lonelyplanet.com/spain/aragon-basq...
6,A foodie tour of Catalonia,0.130252,-0.094497,-0.005959,0.029796,https://lonelyplanetwp.imgix.net/2016/12/IMG_2...,https://www.lonelyplanet.com/spain/catalonia/t...
7,Barcelona’s surreal world of art,0.136020,-0.103807,-0.006047,0.026166,http://static1.squarespace.com/static/5779ddf6...,https://www.lonelyplanet.com/spain/barcelona/t...
8,"Hiking the Alpujarras, southern Spain’s less-t...",0.078767,-0.088990,-0.006168,-0.016391,https://lonelyplanetwp.imgix.net/2016/03/misty...,https://www.lonelyplanet.com/spain/andalucia/l...
9,Tapas: the essential guide to bar food in Spain,0.071252,-0.116082,-0.005930,-0.050759,https://lonelyplanetwp.imgix.net/2012/08/tapas...,https://www.lonelyplanet.com/spain/travel-tips...


In [167]:
x = df_final['title'].iloc[:5].tolist()
x

['Madrid and the art of tapas',
 "Andalucía's local hangout: Costa de la Luz",
 'An introduction to Catalan cuisine',
 'Galicia: exploring the crossroads of Spanish and Celtic culture',
 'Architectural sights of Andalucía']

In [168]:
chee = df_finalFinal.to_json(orient='records')

In [169]:
df_as_json = df_finalFinal.to_dict(orient='records')

In [170]:
import json
json.dumps(json.loads(df_finalFinal.reset_index().to_json(orient='records')))

'[{"index": 0, "title": "Madrid and the art of tapas", "similarity_firstP": 0.3209998995, "similarity_body": -0.0672250166, "similarity_title": -0.0060595331, "final_score": 0.2477153499, "image_one": "https://fthmb.tqn.com/NQWHLZNDNQ-z6d2bFjOhEa7_jmo=/960x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/475689807-56a8d2f95f9b58b7d0f57954.jpg", "URL": "https://www.lonelyplanet.com/spain/madrid/travel-tips-and-articles/madrid-and-the-art-of-tapas/40625c8c-8a11-5710-a052-1479d2773e92"}, {"index": 1, "title": "Andaluc\\u00eda\'s local hangout: Costa de la Luz", "similarity_firstP": 0.2813934565, "similarity_body": -0.0755304401, "similarity_title": -0.0059391591, "final_score": 0.1999238572, "image_one": "http://static1.squarespace.com/static/5779ddf6d1758e240176dffd/589d0b1d20099e45235145e4/589d0bf2bf629a9c5e66adca/1519679096230/iStock-522001766.jpg?format=1000w", "URL": "https://www.lonelyplanet.com/spain/andalucia/huelva-s-costa-de-la-luz/travel-tips-and-articles/andal